# Homework 4 - pandas (YOUR NAME HERE)  
### due 3/14/2022   

Use this notebook and prompts to complete the homework. Throughout there will be hints and some code provided  

### Things you will need:  
- Install os, NumPy, pandas  
- states_covid.csv  
- Bloom_etal_2018_Reduced_Dataset.csv  
- logfiles.tgz (or some other multiple file dataset)  

*NOTE*: Make sure your PATH is correct  

**import packages & check required datasets**   

In [52]:
import os
import numpy as np
import pandas as pd

In [53]:
PATH = '/Users/abbymiller/Desktop/BIOL_792/Data_Science_For_Biology_II/Data_Science_For_Biology_II/Part.3.PythonProgramming/Pandas/'

In [54]:
assert os.path.exists(os.path.join(PATH,'states_covid.csv')), 'states_covid.csv does not exist' 

In [55]:
assert os.path.exists(os.path.join(PATH,'Bloom_etal_2018_Reduced_Dataset.csv')), 'Bloom_etal_2018_Reduced_Dataset.csv does not exist'

## Task 1 - DataFrame manipulation  

Using **states_covid.csv**, we are going to read the data in as a DataFrame to manipulate, subset, and filter in various ways.   

### Task 1.1 

Read in states_covid.csv with date as a "date" dtype, and only columns consisting of the hospitalization (4 col), ICU (2 col), and Ventilators (2 col)  

In [56]:
state_covid_sub_df = pd.read_csv('states_covid.csv',usecols=['date', 'state','hospitalized', 'hospitalizedCumulative','hospitalizedCurrently', 'hospitalizedIncrease','inIcuCumulative','inIcuCurrently','onVentilatorCumulative', 'onVentilatorCurrently'],parse_dates=['date'],infer_datetime_format=True)
state_covid_sub_df.head()

,date,state,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,inIcuCumulative,inIcuCurrently,onVentilatorCumulative,onVentilatorCurrently
0,2021-02-23,AK,1260.0,1260.0,38.0,9,NaN,NaN,NaN,5.0
1,2021-02-23,AL,45250.0,45250.0,762.0,122,2632.0,NaN,1497.0,NaN
2,2021-02-23,AR,14617.0,14617.0,545.0,47,NaN,204.0,1505.0,99.0
3,2021-02-23,AS,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
4,2021-02-23,AZ,57072.0,57072.0,1515.0,78,NaN,447.0,NaN,266.0


### Task 1.2 

For each of the following catergories: *currently* hospitalized, *currently* in the ICU, and *currently* on ventilation...  
Find the 5 states with the greatest numbers in each catergory and list them in order.      
*hint*: sort_values, unique  

In [57]:
state_covid = state_covid_sub_df.groupby('state',as_index=True).agg(['sum'])
state_covid.columns = state_covid.columns.map('_'.join)

hospital_covid = state_covid.sort_values('hospitalizedCurrently_sum',ascending=False)
print(hospital_covid.head(5))

icu_covid = state_covid.sort_values('inIcuCurrently_sum',ascending=False)
print(icu_covid.head(5))

ventilation_covid = state_covid.sort_values('onVentilatorCurrently_sum',ascending=False)
print(ventilation_covid.head(5))

       hospitalized_sum  hospitalizedCumulative_sum  \
state                                                 
CA                  0.0                         0.0   
TX                  0.0                         0.0   
NY           28486669.0                  28486669.0   
FL           12473211.0                  12473211.0   
IL                  0.0                         0.0   

       hospitalizedCurrently_sum  hospitalizedIncrease_sum  \
state                                                        
CA                     2588299.0                         0   
TX                     2062892.0                         0   
NY                     1550375.0                     89995   
FL                     1106003.0                     79410   
IL                      957999.0                         0   

       inIcuCumulative_sum  inIcuCurrently_sum  onVentilatorCumulative_sum  \
state                                                                        
CA                     

/var/folders/sy/12_032f93b17b4n28cgmrhzc0000gn/T/ipykernel_58989/1420535146.py:1: FutureWarning: ['date'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  state_covid = state_covid_sub_df.groupby('state',as_index=True).agg(['sum'])


### Task 1.3 

Find the date in which each state crossed 1000 cumulative hospitilized covid patients

In [58]:
querystate = state_covid_sub_df.query("hospitalizedCumulative >= 1000")[['date','state']]
print(querystate.groupby('state').agg('min'))

            date
state           
AK    2020-12-29
AL    2020-05-01
AR    2020-06-15
AZ    2020-04-11
CO    2020-04-07
CT    2020-05-01
FL    2020-04-02
GA    2020-04-02
HI    2020-10-18
ID    2020-08-11
IN    2020-05-08
KS    2020-06-17
KY    2020-04-18
MA    2020-04-04
MD    2020-04-06
ME    2020-12-23
MN    2020-04-30
MS    2020-04-26
MT    2020-10-17
ND    2020-10-08
NE    2020-06-11
NH    2021-01-24
NJ    2020-05-26
NM    2020-05-20
NY    2020-03-21
OH    2020-04-04
OK    2020-06-03
OR    2020-06-25
PA    2020-04-04
RI    2020-05-03
SC    2020-04-29
SD    2020-08-29
TN    2020-04-29
UT    2020-06-13
VA    2020-04-16
WA    2020-05-21
WI    2020-04-14
WY    2020-12-18


## Task 2 - DataFrame summarizing  

Using **Bloom_etal_2018_Reduced_Dataset.csv**, we are going to do more dataframe manipulation and subsetting and summarizing    

### Task 2.1 

Read in Bloom_etal_2018_Reduced_Dataset.csv and create two new columns ('genus','species') that consists of the column *taxa* split at the underscore. Print out the head of this new dataframe and the number of unique genera**     

*hint:* pd.str.split(,expand=True)  

for example:  

| taxa | genus | species   
| :------ | :-- | :---   
| Alosa_alabamae | Alosa | alabamae  

In [88]:
Bloom_df = pd.read_csv('Bloom_etal_2018_Reduced_Dataset.csv') #read in csv
Bloom_df[['genus','species']] = Bloom_df.taxa.str.split(pat='_',n=1,expand=True)
Bloom_df.head()

,taxa,logbodysize,trophic_position,Reg,genus,species
0,Alosa_alabamae,1.707570,0.431364,diadromous,Alosa,alabamae
1,Alosa_alosa,1.778151,0.556303,diadromous,Alosa,alosa
2,Alosa_fallax,1.778151,0.556303,diadromous,Alosa,fallax
3,Alosa_mediocris,1.778151,0.612784,diadromous,Alosa,mediocris
4,Alosa_pseudoharengus,1.602060,0.544068,diadromous,Alosa,pseudoharengus


### Task 2.2 

Create a new dataframe with the mean *logbodysize* and *trophicposition* of each genera. Sort this data frame by the largest body size. Print the head of this dataframe.

In [90]:
New_bloom_df = Bloom_df.groupby('genus',as_index=True).agg(['mean'])
New_bloom_df.head()
New_bloom_df.index
New_bloom_df.columns

New_bloom_df.columns = New_bloom_df.columns.map('_'.join)
New_bloom_df

body_df = New_bloom_df.sort_values('logbodysize_mean',ascending=False)
body_df.head()

body_df.head() 
body_df.tail()

/var/folders/sy/12_032f93b17b4n28cgmrhzc0000gn/T/ipykernel_58989/3792822266.py:1: FutureWarning: ['taxa', 'Reg', 'species'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  New_bloom_df = Bloom_df.groupby('genus',as_index=True).agg(['mean'])


,logbodysize_mean,trophic_position_mean
genus,,
Tenualosa,1.778151,0.462398
Alosa,1.739062,0.540815
Coilia,1.544068,0.477121
Ethmalosa,1.544068,0.397940
Potamalosa,1.505150,0.518514


Which genera is the smallest and largest?

In [ ]:
#largest = Tenaulosa
#smallest = Amazonsprattus

What is the trophic position of the smallest and largest?  

In [ ]:
#largest trophic = 0.46
#smallest trophic = 0.53

## Task 3 - Read in muliple files to a dictionary and make a DataFrame - **OPTIONAL/BONUS**  

### This is not something you are expected to do in this course, but just here to give you an idea of the things that you COULD do. Answers will be posted after due date.  


Using **logfiles**: we are going to do read in each file, get some data, append it to a dictionary to later make into a dataframe.     

**note:** *make sure to unzip logfiles*

In [20]:
cd $PATH

/Users/trevorfaske/Desktop/Classes/DataScience/pandas/Spring22


In [21]:
!tar -xzf logfiles.tgz #unzip logfiles

In [22]:
log_dir = os.path.join(PATH,'logfiles')
assert os.path.exists(log_dir), 'log_dir does not exist'

First step is to find the necessary files. The number of files in the log files is 36, make sure you have that many as well  

In [23]:
!ls -l logfiles/*txt | wc -l 

      36


In [24]:
logfiles = !find $log_dir -name '*txt' #unix command to find files in log_dir directory
logfiles = [os.path.abspath(x) for x in logfiles] #this finds the full path to the file
print(logfiles[1])

/Users/trevorfaske/g/Classes/DataScience/pandas/Spring22/logfiles/1901302121_H8_S_14.txt.txt


In [25]:
assert(len(logfiles)==36), 'Do not have correct number of logfiles'

### Getting a little tricky here  

Read in each of the logfiles, for each file extract:  
- minimum temperature  
- maximum temperature  
- date of minimum temp   
- date of maximum temp   
- mean temp for each file.   

This data should all be appended for a dictionary within a for loop:    
Key should be the file name without the path or .txt extension  
Values should be (minTemp,maxTemp,minDate,maxDate,meanTemp)

I recommend making this work for one file first, then putting the rest in a for loop to do the rest.  

Below is an example of how to read in one file

*hint:* do not read date in as date object

In [41]:
### set up data frame as you read in each file
infile = pd.read_csv(logfiles[0],sep='\t',engine='python')
infile.columns = ['Index','Date','Time','Temp','Type']
infile = infile[['Date','Time','Temp']]
infile.head()

,Date,Time,Temp
0,9/16/2013,8:00:00 AM,47.9
1,9/16/2013,8:35:00 AM,48.2
2,9/16/2013,9:10:00 AM,48.7
3,9/16/2013,9:45:00 AM,49.4
4,9/16/2013,10:20:00 AM,50.3


Do everything in steps, make sure it works. Calculate summaries with this one infile:

In [26]:
minTemp =   
maxTemp =  
minDate = infile['Date'][infile['Temp'] == infile['Temp'].min()].unique()[0] #use this for minDate 
maxDate = infile['Date'][infile['Temp'] == infile['Temp'].max()].unique()[0] #use this for maxDate 
meanTemp = 

SyntaxError: invalid syntax (<ipython-input-26-d888355c2242>, line 1)

To get you started, I suggest writing some dummy code in plain words to help outline your for loop:  

In [ ]:
logfiles_dict = {}  
for f in logfiles:  
    #do something   
    #do not read date in as date object  
    #do more something   
    #do other stuff  
    #make print statements EVERYWHERE  
    #append to dict  
    #blahbahblah

Then do the real code below here. You don't need to turnin your thoughts. Just put it in there as a help reminder. Most people all still do this, no matter how advanced they are

**Once you have created a DataFrame with all the logfiles, print the head and save it to an outfile using pd.to_csv() as logfiles_df.csv** 

#### Here is an example of the final product

In [32]:
logfiles_df.head()

,minTemp,maxTemp,minDate,maxDate,meanTemp
1901302235_H6_S_14,21.3,65.8,12/6/2013,7/13/2014,35.726191
1901302121_H8_S_14,17.9,73.4,10/28/2013,7/13/2014,34.698866
1901302217_H18_D_14,21.0,82.4,10/28/2013,7/26/2014,33.334564
1901302110_H13_S_14,22.8,60.9,10/4/2013,7/13/2014,32.240032
1901302146_H14_D_14,24.6,66.1,1/21/2014,7/13/2014,34.242075
